In [1]:
# importing libraries
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import h5py
from zipfile import ZipFile


#checkpoint_dir = 'C://Users//TOSHIBA//Downloads//checkpoint_final-20200722T030914Z-001.zip' # directory of model or checkpoint
#print(os.listdir(checkpoint_dir))

In [2]:
import h5py
cv2.__version__

'4.3.0'

In [3]:
model =tf.keras.models.load_model('C://Users//TOSHIBA//Downloads//mask_model.h5') #loading the model

In [4]:
labels ={0 :' mask', 1 :'No Mask'}
color = {1 : (0,0,255),0 :(0,255,0)}

# Testing on Images

In [42]:
import os
dir = 'C://Users//TOSHIBA//Downloads//mask_images//'
for images in os.listdir(dir) :
    print(dir+images)
    img = cv2.imread(dir+images)
    face_clsfr=cv2.CascadeClassifier('C://Users//TOSHIBA//Downloads//haarcascade_frontalface_default.xml')
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)
    for x,y,w,h in faces :
                face_img=img[y:y+h,x:x+w]
                resized=cv2.resize(face_img,(224,224))

                normalized=resized/255.0
                reshaped=np.reshape(normalized,(1,224,224,3))

                result=model.predict(reshaped)
                label=np.argmax(result,axis=1)[0]

                cv2.rectangle(img,(x,y),(x+w,y+h),color[label],2)
                cv2.rectangle(img,(x,y-40),(x+w,y),color[label],-1)
                cv2.putText(img, labels[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('output',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()




C://Users//TOSHIBA//Downloads//mask_images//mask.jpg
C://Users//TOSHIBA//Downloads//mask_images//mask_multi.jpg
C://Users//TOSHIBA//Downloads//mask_images//wiyhout.jpg


# Live video code

In [19]:
# Code for live video streaming

#def callback(x) :
#    pass

#cv2.namedWindow('Control Window')
#cv2.createTrackbar('Threshold','Control Window', 70, 200, callback)

face_clsfr=cv2.CascadeClassifier('C://Users//TOSHIBA//Downloads//haarcascade_frontalface_default.xml')
#mouth_classifier =cv2.CascadeClassifier('C://Users//TOSHIBA//Downloads//Mouth.xml')
#Avoid mouth classifier

source=cv2.VideoCapture('C://Users//TOSHIBA//Downloads//mask.mp4') #directory to video
#source = cv2.VideoCapture(0) #webcam feed

#threshold = cv2.getTrackbarPos('Threshold', 'Control Window')


In [20]:
while(source.isOpened()):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    

    thres, b_w =  cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5) 
    faces_b_w=face_clsfr.detectMultiScale(b_w,1.5,5)
    
    if len(faces) ==0 and len(faces_b_w) == 0 :
        cv2.putText(img,'No face found', (30,30),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    elif len(faces) == 0 and len(faces_b_w) ==1 :
        label = 0
        cv2.putText(img,labels[label], (30, 30),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    else :
        for (x,y,w,h) in faces:
            face_img=img[y:y+h,x:x+w]
            resized=cv2.resize(face_img,(224,224))
        
            normalized=resized/255.0
            reshaped=np.reshape(normalized,(1,224,224,3))
            
            
            result=model.predict(reshaped)
            label=np.argmax(result,axis=1)[0]
            probs = round((np.max(result)*100),2)
      
            cv2.rectangle(img,(x,y),(x+w,y+h),color[label],2)
            cv2.rectangle(img,(x,y-40),(x+w,y),color[label],-1)
            cv2.putText(img, labels[label]+str(probs), (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    cv2.imshow('face',face_img)
    
    
    key=cv2.waitKey(1)
    
    if(key==27):
        break
cv2.destroyAllWindows()
source.release()

